In [1]:
import os
from pathlib import Path
from itertools import zip_longest, cycle

FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day17.txt'

# Part One

In [2]:
def left(a):
    m = max(a)
    if m << 1 > 127:
        return a
    else:
        return [i << 1 for i in a]
    
def right(a):
    if any(i % 2 for i in a):
        return a
    else:
        return [i >> 1 for i in a]

def print_block(b):
    for i in reversed(b):
        print('{0:b}'.format(i).zfill(7).replace('0', '.').replace('1', '#'))

        
jet = {
    '<': left,
    '>': right
}

with open(FOLDER / in_file) as f:
    data = f.read()


    
blocks = [
    [30],
    [8, 28, 8],
    [28, 4, 4],
    [16, 16, 16, 16],
    [24, 24] 
]


def drop(block, room, jets):
    # index top of stack
    top = len(room)
    # position of bottom of block
    pos = top + 3
    while True:
        overlap = pos - top
      
        jet = next(jets)
        moved_block = jet(block)

        # only jet over it there's no collision
        if overlap >=0 or not any(r & b for b, r in zip(moved_block[:-overlap], room[overlap:])):
            block = moved_block

        if pos <= 0:
            break

        if overlap <= 0 and any(r & b for r, b in zip(block[:-(overlap -1)], room[overlap-1:])):

            break
    
        pos -= 1
    
    

    if overlap < 0:
        block_overlap, block = block[:-overlap], block[-overlap:]
        room_overlap = room[overlap:]

        room[overlap:] = [r | b for r, b in zip_longest(room_overlap, block_overlap, fillvalue=0)]
    
    room.extend(block)
    


room = []
block_cycle = cycle(blocks)
jets = cycle(jet[c] for c in list(data))

n = 2022
for i in range(n):
    drop(next(block_cycle), room, jets)
    
len(room)

3177

# Part 2

In [3]:
with open(FOLDER / in_file) as f:
    data = f.read()
    
room = []
block_cycle = cycle(blocks)
jets = cycle(jet[c] for c in list(data))

n = 8000
for i in range(n):
    drop(next(block_cycle), room, jets)
    

def find_cycle_length(l):
    ''' 
    Shift the rows until a lot of stuff matches up
    assume a really good match represents a cycle
    '''
    highest_matches = 0
    best_shift = 0
    for i in range(1, len(l)):
        same = sum(a == b for a, b in zip(l, l[i:]))
        if same > highest_matches:
            highest_matches = same
            best_shift = i
    return best_shift       

cycle_rows = find_cycle_length(room)
print(f"rows per cycle: {cycle_rows}")

for i in range(7999):
    if all(a == b for a, b in zip(room[i:], room[i+cycle_rows:])):
        prefix_rows = i
        print("prefix_length:", i)
        break
        
prefix_rows, cycle_rows

rows per cycle: 2724
prefix_length: 264


(264, 2724)

In [4]:
# Gut check: do they sync for a reasonable length?
# take slice starting after prefix and after prefix + multiple of cycle
# these should be the same if the cycle prefix is correct

room[prefix_rows:prefix_rows+2000] == room[prefix_rows + cycle_rows:prefix_rows + cycle_rows+2000]

True

In [5]:
# Try to determine how many blocks create the above number of rows

room = []
block_cycle = cycle(blocks)
jets = cycle(jet[c] for c in list(data))

n = 5000
prefix_blocks = 0
cycle_blocks = 0

for i in range(n):
    drop(next(block_cycle), room, jets)
    if len(room) == prefix_rows:
        prefix_blocks = i + 1
    if len(room) == prefix_rows + cycle_rows:

        cycle_blocks = i + 1
        
cycle_blocks = cycle_blocks - prefix_blocks
print(f"Prefix rows are made of {prefix_blocks} blocks")
print(f"A complete cycle is made of {cycle_blocks} blocks")

prefix_blocks, cycle_blocks

Prefix rows are made of 175 blocks
A complete cycle is made of 1740 blocks


(175, 1740)

In [6]:
blocks_after_prefix = (1000000000000 - prefix_blocks)

# This number of blocks does not represent an even cycle :(
# Figure out remained then figure out how many rows are in the first
# part of a cycle of remaining_block blocks

even_cycles, remaining_block = divmod(blocks_after_prefix, cycle_blocks)

even_cycles, remaining_block

(574712643, 1005)

In [7]:
# find length of first 1005 blocks in a cycle

room = []
block_cycle = cycle(blocks)
jets = cycle(jet[c] for c in list(data))

n = prefix_blocks
for i in range(n):
    drop(next(block_cycle), room, jets)
    
pre_len = len(room)

for i in range(remaining_block):
    drop(next(block_cycle), room, jets)


remmaining_rows = len(room) - pre_len

# answer:
prefix_rows + even_cycles * cycle_rows + remmaining_rows

1565517241382